In [1]:
import os

In [2]:
pwd


'd:\\PW_DS\\Deep Learning\\End-To-End-Chicken-Disease-Classification-Project-using-CNN\\research'

In [3]:
os.chdir('../')

In [4]:
pwd


'd:\\PW_DS\\Deep Learning\\End-To-End-Chicken-Disease-Classification-Project-using-CNN'

## ---

In [ ]:
### ------------

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

## Entity

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int



## Config Manager



In [8]:
from chicken_disease_classification.constants import  *
from chicken_disease_classification.utils.common import  read_yaml, create_directories

In [10]:
class ConfigurationManger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        
        eval_config = EvaluationConfig(
            path_of_model = 'artifacts/training/model.h5',
            training_data = 'artifacts/data_ingestion/chicken-disease-dataset/poultry_diseases',
            all_params = self.params,
            params_image_size = self.params.vgg_16.IMAGE_SIZE,
            params_batch_size = self.params.vgg_16.BATCH_SIZE
        )

        return eval_config  


## Components

In [16]:
from urllib.parse import urlparse
from chicken_disease_classification.utils.common import load_json, save_json
import tensorflow as tf

In [19]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    

    def valid_generator(self):

        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30 
        )

        data_flow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )


        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            class_mode='categorical',
            **data_flow_kwargs

        )


    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.valid_generator()
        self.score = model.evaluate(self.valid_generator)


    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path('scores.json'), data=scores)
        


## Pipeline

In [20]:
try:
    config = ConfigurationManger()
    eval_config = config.get_evaluation_config()
    eval = Evaluation(config = eval_config)
    eval.evaluation()
    eval.save_score()

except Exception as e:
    raise e

[2024-01-17 19:58:57,336: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-01-17 19:58:57,341: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-01-17 19:58:57,343: INFO: common: Created Directory at artifacts.]
Found 1845 images belonging to 3 classes.
116/116 [==============================] - 259s 2s/step - loss: 0.7658 - accuracy: 0.9144
[2024-01-17 20:03:19,667: INFO: common: json file saved at: scores.json.]
